# preset  

In [29]:
#preset1 - 패키지 opt1 - 구글드라이브에 설치하기
import os, sys
from google.colab import drive

#preset1 -  Colab Notebooks 안에 .env 폴더에 패키지 저장 ( 심볼링 링크가 작동 안된다. )
"""
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/_ColabNotebooks/.env', my_path) # 기존 path를 바꿔치기
sys.path.insert(0, my_path)
"""
#preset1 -  Dependency - 위 preinstall 작업을 안하면, 세션 할당시 매번 install
"""
!pip install --target=$my_path finance_datareader
"""

#preset1 - 패키지 opt2 - 매번 설치하기

!pip install finance_datareader

#preset -  my_data있는 드라이브 마운트
drive.mount('/content/drive')
prefix_path = "/content/drive/My Drive/_ColabNotebooks/"


import pandas as pd
import numpy as np

#preset - 주피터 노트북 여러줄 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#preset - pandas
pd.set_option("display.float_format",lambda x: '%.3f'%x) # 소수점 6자리 > 3자리만 보여달라
pd.set_option("max_columns",None) # 컬럼을 ... 으로 요약하지 마라 ( row는 요약되어있다.)
# pd.DataFrame(np.random.randn(1000,300))

"\ndrive.mount('/content/drive')\nmy_path = '/content/notebooks'\nos.symlink('/content/drive/My Drive/_ColabNotebooks/.env', my_path) # 기존 path를 바꿔치기\nsys.path.insert(0, my_path)\n"

'\n!pip install --target=$my_path finance_datareader\n'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# DataFrame에 data row 추가하기

## `loc[]`을 사용하여 추가하기

In [30]:
df = pd.DataFrame(columns=['a', 'b'])
df.head()

,a,b


### Add data as 'list'

In [31]:
# df.loc[0] # get operation
df.loc[0] = [1, 2]  # set operation
df.head()

,a,b
0,1,2


In [32]:
df.loc['ㅋㅋ'] = [1, 2]
df.head()

,a,b
0,1,2
ㅋㅋ,1,2


### Add data as 'dict'

In [33]:
df.loc[len(df)] = {'b' : 'ㅎ', 'a': 'ㅋ'}
df.head()

,a,b
0,1,2
ㅋㅋ,1,2
2,ㅋ,ㅎ


### Add data as 'Series'

In [34]:
df.loc["yay"] = pd.Series({'a': 'ㅋ', 'b' : 'ㅎ'})
df.tail()

,a,b
0,1,2
ㅋㅋ,1,2
2,ㅋ,ㅎ
yay,ㅋ,ㅎ


In [35]:
# 이미 존재한는 index에 넣기 
df.loc["yay"] = pd.Series({'a': '1111', 'b' : '2222'})
df.tail()

,a,b
0,1,2
ㅋㅋ,1,2
2,ㅋ,ㅎ
yay,1111,2222


- 위 방법들은 다 inplace 방식  (Upsert)
  * 존재하면 덮어 씌운다.
  * 존재하지 않으면 새로 만든다.   

- 기존의 데이터가 손상된다.(결국!)


## `append()` 사용하여 추가하기

- 위의 `loc`과는 다르게 not in-place(returns a new copy of the DataFrame)

- `append()` : it only accecpt
    - DataFrame
    - Series
    - Dictionary
    - list of these(**Not `list` itself**)

In [36]:
names_df = pd.DataFrame(
    {
        'Name':['철수', '영희', '영수', '영미'],
        'Age':[12, 13, 14, 15]
    },
    index = ['Canada', 'Canada', 'USA', 'USA']
)
names_df

,Name,Age
Canada,철수,12
Canada,영희,13
USA,영수,14
USA,영미,15


In [37]:
# Error (에러내용 확인!) => index를 뭐로 설정해야될지 모르기 때문
# names_df.append(
#     {'Name':'명수', 'Age':1}
# )

### `ignore_index=True`

- 이전 index를 다 reset한다

In [38]:
names_df.append( 
    {'Name':'명수', 'Age':100},
    ignore_index=True
)

,Name,Age
0,철수,12
1,영희,13
2,영수,14
3,영미,15
4,명수,100


In [39]:
# 리스트 of 딕셔너리로도 전달 가능
names_df.append(
    [
        {'Name':'명수', 'Age':1},
        {'Name':'동수', 'Age':2}
    ], 
    ignore_index=True
)

,Name,Age
0,철수,12
1,영희,13
2,영수,14
3,영미,15
4,명수,1
5,동수,2


In [40]:
# append()는 내부적으로 copy()를 하기 때문에, 원본은 그대로 유지
names_df

,Name,Age
Canada,철수,12
Canada,영희,13
USA,영수,14
USA,영미,15


### Original index 유지하기 => `append()` 할 때, `dict`대신에 `Series`를 전달하면 됨

- 참고: `Series`를 `append`를 할 때는, `Series`의 index가 target DataFrame의 column이 되고, name이 target DataFrame의 index가 됨

In [41]:
# 특이하게 index가 df의 index에 매핑되는것이 아닌 컬럼에 매핑 된다. ✅
# `name` arg를 통해서 Series의 name을 부여하기
s = pd.Series({'Name': 'Zach', 'Age': 3}, name=len(names_df))
s

Name    Zach
Age        3
Name: 4, dtype: object

In [42]:
names_df.append(s)

,Name,Age
Canada,철수,12
Canada,영희,13
USA,영수,14
USA,영미,15
4,Zach,3


In [43]:
# 리스트 of Series로도 전달 가능
s1 = pd.Series({'Name': 'Zach', 'Age': 3}, name=len(names_df))
s2 = pd.Series({'Name': 'Zayd', 'Age': 2}, name='USA')
names_df.append([s1, s2])
# 결과적으로 row를 추가하게 된다.

,Name,Age
Canada,철수,12
Canada,영희,13
USA,영수,14
USA,영미,15
4,Zach,3
USA,Zayd,2


- 참고: Series의 `name`은 어떤 operation을 하느냐에 따라서, index or column이 될 수 있음

In [44]:
# 시리즈의 인덱스가 df의 인덱스에 매핑될 수 있다!
# 컬럼 증가
pd.concat([s1, s2], axis=1)

,4,USA
Name,Zach,Zayd
Age,3,2


# concat, join, and merge 

## `concat()` 

- DataFrame or Series object를 수직적으로 or 수평적으로 '연결'
- **index(or columns)**에 대해 algin (not values)
- Defaults to `outer` join
    - operation axis에 따라 concat되는 object의 column or index가 union됨

- 예제

In [45]:
import FinanceDataReader as fdr

In [46]:
samsung_df = fdr.DataReader('005390', '2009-01-01', '2017-12-31')
kodex_df = fdr.DataReader('069500', '2016-01-01', '2017-12-31')

In [47]:
samsung_df.head(2)
kodex_df.head(2)

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2009-01-02,200,234,200,234,2122,0.083
2009-01-05,229,234,213,223,6890,-0.047


,Open,High,Low,Close,Volume,Change
Date,,,,,,
2016-01-04,21890,21899,21516,21525,6928106,-0.020
2016-01-05,21474,21728,21474,21660,12419517,0.006


In [48]:
pd.concat(
    [samsung_df, kodex_df]
) 

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2009-01-02,200,234,200,234,2122,0.083
2009-01-05,229,234,213,223,6890,-0.047
2009-01-06,210,234,210,212,2144,-0.049
2009-01-07,214,220,208,216,2434,0.019
2009-01-08,216,220,200,215,2280,-0.005
...,...,...,...,...,...,...
2017-12-21,30523,30579,30002,30006,8563000,-0.019
2017-12-22,30085,30216,30010,30151,8482482,0.005
2017-12-26,30213,30395,30024,30029,7603311,-0.004


- Column, Index alignment 특징은 그대로 적용됨!

In [49]:
kodex_df[['Open', 'Close']].head()

,Open,Close
Date,,
2016-01-04,21890,21525
2016-01-05,21474,21660
2016-01-06,21651,21503
2016-01-07,21449,21297
2016-01-08,21166,21448


In [50]:
pd.concat(
    [
        samsung_df, 
        kodex_df[['Open', 'Close']]
    ]
).tail(2)   # head()도 한번 실행해보세요!

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2017-12-27,30127,nan,nan,30381,nan,nan
2017-12-28,30391,nan,nan,30763,nan,nan


- `keys`, `names` args

In [51]:
pd.concat(
    [samsung_df, kodex_df],
    keys=['삼성', 'KODEX200'], 
).head()

Open  High  Low  Close  Volume  Change
   Date                                              
삼성 2009-01-02   200   234  200    234    2122   0.083
   2009-01-05   229   234  213    223    6890  -0.047
   2009-01-06   210   234  210    212    2144  -0.049
   2009-01-07   214   220  208    216    2434   0.019
   2009-01-08   216   220  200    215    2280  -0.005

In [52]:
pd.concat(
    [samsung_df, kodex_df],
    keys=['삼성', 'KODEX200'],
    names=['종목명']
).head()

Open  High  Low  Close  Volume  Change
종목명 Date                                              
삼성  2009-01-02   200   234  200    234    2122   0.083
    2009-01-05   229   234  213    223    6890  -0.047
    2009-01-06   210   234  210    212    2144  -0.049
    2009-01-07   214   220  208    216    2434   0.019
    2009-01-08   216   220  200    215    2280  -0.005

In [53]:
pd.concat(
    [samsung_df, kodex_df],
    keys=['삼성', 'KODEX200'],
    names=['종목명', '날짜']
).head()

Open  High  Low  Close  Volume  Change
종목명 날짜                                                
삼성  2009-01-02   200   234  200    234    2122   0.083
    2009-01-05   229   234  213    223    6890  -0.047
    2009-01-06   210   234  210    212    2144  -0.049
    2009-01-07   214   220  208    216    2434   0.019
    2009-01-08   216   220  200    215    2280  -0.005

- On `axis` = 1  

In [54]:
pd.concat([samsung_df, kodex_df], axis=1).head()

,Open,High,Low,Close,Volume,Change,Open,High,Low,Close,Volume,Change
Date,,,,,,,,,,,,
2009-01-02,200,234,200,234,2122,0.083,nan,nan,nan,nan,nan,nan
2009-01-05,229,234,213,223,6890,-0.047,nan,nan,nan,nan,nan,nan
2009-01-06,210,234,210,212,2144,-0.049,nan,nan,nan,nan,nan,nan
2009-01-07,214,220,208,216,2434,0.019,nan,nan,nan,nan,nan,nan
2009-01-08,216,220,200,215,2280,-0.005,nan,nan,nan,nan,nan,nan


In [55]:
pd.concat([samsung_df, kodex_df], keys=['삼성', 'KODEX200'], axis=1).head(2)

삼성                               KODEX200                        \
           Open High  Low Close Volume Change     Open High Low Close Volume   
Date                                                                           
2009-01-02  200  234  200   234   2122  0.083      nan  nan nan   nan    nan   
2009-01-05  229  234  213   223   6890 -0.047      nan  nan nan   nan    nan   

                   
           Change  
Date               
2009-01-02    nan  
2009-01-05    nan

- `join`  argument
    - How to handle **indexes** on **other** axis(es). 즉, concat의 대상이 되는(=명시되는) axis 말고, 다른 axis에 대해 어떻게 join할 것인가

![image.png](attachment:image.png)

In [56]:
# default 'outer' join
pd.concat([samsung_df, kodex_df], keys=['삼성', 'kodex'], axis=1, names=['종목명']).head()

종목명          삼성                               kodex                        \
           Open High  Low Close Volume Change  Open High Low Close Volume   
Date                                                                        
2009-01-02  200  234  200   234   2122  0.083   nan  nan nan   nan    nan   
2009-01-05  229  234  213   223   6890 -0.047   nan  nan nan   nan    nan   
2009-01-06  210  234  210   212   2144 -0.049   nan  nan nan   nan    nan   
2009-01-07  214  220  208   216   2434  0.019   nan  nan nan   nan    nan   
2009-01-08  216  220  200   215   2280 -0.005   nan  nan nan   nan    nan   

종목명                
           Change  
Date               
2009-01-02    nan  
2009-01-05    nan  
2009-01-06    nan  
2009-01-07    nan  
2009-01-08    nan

In [57]:
# join = inner (date intersection)
pd.concat([samsung_df, kodex_df], keys=['삼성', 'kodex'], axis=1, names=['종목명'], join='inner').head()

종목명           삼성                                    kodex                \
            Open  High   Low Close   Volume Change   Open   High    Low   
Date                                                                      
2016-01-04  1470  1485  1430  1430   862249 -0.017  21890  21899  21516   
2016-01-05  1405  1440  1400  1400   844949 -0.021  21474  21728  21474   
2016-01-06  1415  1435  1390  1400  1061873  0.000  21651  21674  21385   
2016-01-07  1405  1415  1350  1350  1184923 -0.036  21449  21529  21237   
2016-01-08  1315  1365  1290  1360   886329  0.007  21166  21460  21119   

종목명                                 
            Close    Volume Change  
Date                                
2016-01-04  21525   6928106 -0.020  
2016-01-05  21660  12419517  0.006  
2016-01-06  21503  10875053 -0.007  
2016-01-07  21297  11213386 -0.010  
2016-01-08  21448  12256496  0.007

In [58]:
# concat 방향이 axis=0이니까, axis=1에 대해서 join이 적용됨
pd.concat([samsung_df.head(), kodex_df[['Close']].head()], join='inner')

,Close
Date,
2009-01-02,234
2009-01-05,223
2009-01-06,212
2009-01-07,216
2009-01-08,215
2016-01-04,21525
2016-01-05,21660
2016-01-06,21503
2016-01-07,21297


- 주의 : `outer` join & column 명이 서로 겹치는 게 없을 때 ! => alignment 가 일치하는게 없으니 NaN으로 메꾼다!

In [59]:
samsung_diff_col_df = samsung_df.copy()
samsung_diff_col_df.columns = ['1_' + col  for col in samsung_df.columns]
samsung_diff_col_df.head()

,1_Open,1_High,1_Low,1_Close,1_Volume,1_Change
Date,,,,,,
2009-01-02,200,234,200,234,2122,0.083
2009-01-05,229,234,213,223,6890,-0.047
2009-01-06,210,234,210,212,2144,-0.049
2009-01-07,214,220,208,216,2434,0.019
2009-01-08,216,220,200,215,2280,-0.005


In [60]:
samsung_df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2009-01-02,200,234,200,234,2122,0.083
2009-01-05,229,234,213,223,6890,-0.047
2009-01-06,210,234,210,212,2144,-0.049
2009-01-07,214,220,208,216,2434,0.019
2009-01-08,216,220,200,215,2280,-0.005


In [61]:
pd.concat([samsung_diff_col_df, kodex_df]).head()

,1_Open,1_High,1_Low,1_Close,1_Volume,1_Change,Open,High,Low,Close,Volume,Change
Date,,,,,,,,,,,,
2009-01-02,200.000,234.000,200.000,234.000,2122.000,0.083,nan,nan,nan,nan,nan,nan
2009-01-05,229.000,234.000,213.000,223.000,6890.000,-0.047,nan,nan,nan,nan,nan,nan
2009-01-06,210.000,234.000,210.000,212.000,2144.000,-0.049,nan,nan,nan,nan,nan,nan
2009-01-07,214.000,220.000,208.000,216.000,2434.000,0.019,nan,nan,nan,nan,nan,nan
2009-01-08,216.000,220.000,200.000,215.000,2280.000,-0.005,nan,nan,nan,nan,nan,nan


### 실전예제: concat을 이용해서 close 데이터만 뽑아내기

In [62]:
total_df = pd.concat([samsung_df, kodex_df], keys=['삼성', 'kodex200'], names=['종목명'])
total_df.head()
total_df.tail()

Open  High  Low  Close  Volume  Change
종목명 Date                                              
삼성  2009-01-02   200   234  200    234    2122   0.083
    2009-01-05   229   234  213    223    6890  -0.047
    2009-01-06   210   234  210    212    2144  -0.049
    2009-01-07   214   220  208    216    2434   0.019
    2009-01-08   216   220  200    215    2280  -0.005

Open   High    Low  Close    Volume  Change
종목명      Date                                                    
kodex200 2017-12-21  30523  30579  30002  30006   8563000  -0.019
         2017-12-22  30085  30216  30010  30151   8482482   0.005
         2017-12-26  30213  30395  30024  30029   7603311  -0.004
         2017-12-27  30127  30376  30066  30381  12654745   0.012
         2017-12-28  30391  30790  30386  30763   7560958   0.013

In [63]:
total_df = total_df.reset_index()
total_df.head()

,종목명,Date,Open,High,Low,Close,Volume,Change
0,삼성,2009-01-02,200,234,200,234,2122,0.083
1,삼성,2009-01-05,229,234,213,223,6890,-0.047
2,삼성,2009-01-06,210,234,210,212,2144,-0.049
3,삼성,2009-01-07,214,220,208,216,2434,0.019
4,삼성,2009-01-08,216,220,200,215,2280,-0.005


In [64]:
total_df.pivot(index='Date', columns='종목명', values='Close')

종목명,kodex200,삼성
Date,,
2009-01-02,nan,234.000
2009-01-05,nan,223.000
2009-01-06,nan,212.000
2009-01-07,nan,216.000
2009-01-08,nan,215.000
...,...,...
2017-12-21,30006.000,1050.000
2017-12-22,30151.000,1050.000
2017-12-26,30029.000,1060.000


- `pivot()` 예시

In [65]:
sample_data = pd.DataFrame(
    {
        "종목명":["삼성", "현대", "하이닉스", "삼성", "현대", "하이닉스"],
        "datetime":["2019-01-01", "2019-01-01", "2019-01-01", "2019-01-02", "2019-01-02", "2019-01-02"],
        "price":[1,2,3, 4,5,6]
    }
)
sample_data

,종목명,datetime,price
0,삼성,2019-01-01,1
1,현대,2019-01-01,2
2,하이닉스,2019-01-01,3
3,삼성,2019-01-02,4
4,현대,2019-01-02,5
5,하이닉스,2019-01-02,6


In [66]:
sample_data.sort_values("종목명")

,종목명,datetime,price
0,삼성,2019-01-01,1
3,삼성,2019-01-02,4
2,하이닉스,2019-01-01,3
5,하이닉스,2019-01-02,6
1,현대,2019-01-01,2
4,현대,2019-01-02,5


In [67]:
sample_data.pivot(index="datetime", columns="종목명", values="price")

종목명,삼성,하이닉스,현대
datetime,,,
2019-01-01,1,3,2
2019-01-02,4,6,5


## `join()`

- 2개의 (보통 index가 다른) dataframe을 하나의 dataframe으로 합칠 때 사용
- Aligns the calling DataFrame's **column(s) or index** with the other DataFrame's **index** 
    1. index - index
    2. columns - index (calling object는 column, called object는 index)
        - `on` arg = calling object의 column
            - called object의 index를 calling object의 "어떤 column"에 맞출것인가 
        - `set_index()` 후,  `on`없이 index-index join과 같은 결과 
- Cartesian product joining
- Defaults to `left` join
- 대부분의 경우 merge랑 호환 가능

- 예제1

In [68]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                      index=['K0', 'K1', 'K2']) 

In [69]:
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                    'D': ['D0', 'D2', 'D3']},
                      index=['K0', 'K2', 'K3'])

In [70]:
left
right

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2


,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [71]:
left.join(right)

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


In [72]:
left.join(right, how='outer')

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


- 예제2

In [73]:
left = pd.DataFrame(
    {
        'A':['A0', 'A1', 'A2', 'A3'],
        'B':['B0', 'B1', 'B2', 'B3'],
        'key':['K0', 'K1', 'K0', 'K1'],
    }
)
right = pd.DataFrame(
    {
        'C':['C0', 'C1'],
        'D':['D0', 'D1'],
    },
    index=['K0', 'K1']
)
left
right

,A,B,key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K0
3,A3,B3,K1


,C,D
K0,C0,D0
K1,C1,D1


- 아래 둘은 결과 같음

In [74]:
# left.join(right, on='key')
left.join(right, on='key').set_index("key")

,A,B,C,D
key,,,,
K0,A0,B0,C0,D0
K1,A1,B1,C1,D1
K0,A2,B2,C0,D0
K1,A3,B3,C1,D1


In [75]:
# left.set_index('key')
left.set_index('key').join(right)

,A,B,C,D
K0,A0,B0,C0,D0
K0,A2,B2,C0,D0
K1,A1,B1,C1,D1
K1,A3,B3,C1,D1


- l_suffix, r_suffix

In [76]:
a = pd.DataFrame([1,2,3], index=['a','b','c'], columns=['안녕'])
b = pd.DataFrame([4,2,6], index=['a','c','d'], columns=['안녕'])
a
b

,안녕
a,1
b,2
c,3


,안녕
a,4
c,2
d,6


In [77]:
a.join(b, lsuffix="_x", rsuffix="_y", how="inner")

,안녕_x,안녕_y
a,1,4
c,3,2


- 예제3 (앞의 lec1\_3에서 median_시가총액 연결하기)

In [78]:
a_df = pd.read_csv(prefix_path+"my_data/Small_and_Big.csv", index_col=[0])
a_df.head()

FileNotFoundError: ignored

In [ ]:
median_df = a_df.groupby(['date']).agg({'시가총액 (보통)(평균)(원)': 'median'})
median_df.columns = ['시가총액_median']
median_df.head()

In [ ]:
joined_df = a_df.join(median_df, on="date")
joined_df.head()

In [ ]:
joined_df[joined_df['date'] == "2000-08-31"].head()

In [ ]:
# Hint: 아래와 같은 느낌으로 하시면 됩니다.
# cond1 = joined_df['시가총액(보통~~)'] <  joined_df['시가총액_median']
# joined_df.loc[cond1, "small_or_big"] = "small"
# joined_df.loc[~cond1, "small_or_big"] = "big"

## `merge()`

- Aligns the calling DataFrame's *column(s)* with the other DataFrame's *column(s)*
    - `left_index`, `right_index` argument도 존재하긴 함(index-index alignment시)
    - `join()`
        - 사실 내부적으로 `reset_index()` 하고 `merge()` 호출함
- Cartesian product joining
- Defaults to `inner` join
- `concat()`과 달리, index, column명이 아니라, value 값 자체를 이용한 join

In [ ]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                        'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3']})
    
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                               'key2': ['K0', 'K0', 'K0', 'K0'],
                                  'C': ['C0', 'C1', 'C2', 'C3'],
                                  'D': ['D0', 'D1', 'D2', 'D3']})

In [ ]:
left
right

In [ ]:
# default: inner join(교집합)
pd.merge(left, right, on=['key1', 'key2'])

In [ ]:
# outer join(합집합)
pd.merge(left, right, how='outer', on=['key1', 'key2'])

In [ ]:
pd.merge(left, right, how='right', on=['key1', 'key2'])

In [ ]:
pd.merge(left, right, how='left', on=['key1', 'key2'])

- More about Cartesian product joining

In [ ]:
left = pd.DataFrame({'A':[1,2,], 'B':[2,2]})
right = pd.DataFrame({'A':[4,5,6], 'B':[2,2,2]})
left
right

In [ ]:
# left, right, inner, outer 결과가 다 같음
pd.merge(left, right, on="B", how='left')

- 예제

In [ ]:
close_df = samsung_df['Close'].reset_index()
vol_df = samsung_df['Volume'].reset_index()

In [ ]:
close_df.head()
vol_df.head()

In [ ]:
vol_df.iloc[:2]

In [ ]:
# default is 'inner' join
pd.merge(close_df, vol_df.iloc[:2])   # 알아서 같은 column 이름끼리 맞춤

In [ ]:
# 'outer' join
pd.merge(close_df, vol_df.iloc[:2], how="outer").head(5)

## join & merge 각각 언제 사용?

- index가 하나라도 관여하면 => `join()`
- 둘다 colum에 맞춰야하면 => `merge()`
- `merge()` 사용시, `left_index`, `right_index` 사용하면 `join()`과 결과가 같음
- `join()` 사용시 `reset_index()`하고, `merge()` 써도 됨

In [ ]:
a = pd.DataFrame([1,2,3], index=['a','b','c'], columns=['안녕'])
b = pd.DataFrame([4,2,6], index=['a','c','d'], columns=['안녕'])
a
b

In [ ]:
a.merge(b)

In [ ]:
a.reset_index().merge(b.reset_index())

In [ ]:
a.merge(b, left_index=True, right_index=True)

In [ ]:
a.join(b, lsuffix="_x", rsuffix="_y", how="inner")

### Concat과 join,merge와의 차이

In [ ]:
a = pd.DataFrame({"a": [1,2,3],}, index=[1,2,3])
b = pd.DataFrame({"b": [1,4,5],}, index=[1,4,5])
a
b

In [ ]:
pd.concat([a, b], axis=1)

In [ ]:
a = pd.DataFrame({"a": [1,2,3],}, index=[1,2,2])
b = pd.DataFrame({"b": [1,4,5],}, index=[1,4,5])
a
b

In [ ]:
# error 발생! => concat()을 cartesian product가 불가능하기 때문에, 중복 index 혹은 column이 있는 경우 작동하지 못함
pd.concat([a, b], axis=1)

# 실전예제

## flipkart

In [ ]:
product_df = pd.read_csv("my_data/product.csv", index_col=0)
review_df = pd.read_csv("my_data/review.csv", index_col=0)

In [ ]:
product_df.shape
review_df.shape

In [ ]:
product_df.head(2)
review_df.head(2)

In [ ]:
flipkart_df = pd.merge(
    product_df, 
    review_df, 

    left_on="id",
    right_on='product__id',
    
    how='right',    # Review에 있는 id, 즉 product__id를 기준으로 데이터를 생성합니다. 만약 "product" 정보가 반드시 존재하는 review들로만 데이터를 구성하고 싶으면 "left"로 하시면 됩니다.
)  

In [ ]:
flipkart_df.shape
flipkart_df.head(2)

In [ ]:
# column을 제거 하기 위해서는 drop([컬럼1, 컬럼2, ..], axis=1)과 같은 방식으로 진행합니다
flipkart_df = flipkart_df.drop(['id', 'product__id', 'author'], axis=1)
flipkart_df.head(2)

## Amazon

In [ ]:
amazon_df = pd.read_csv("my_data/amazon_review1.csv", index_col=0)
amazon_df.head(2)

## 데이터 합치기

In [ ]:
amazon_df.shape
flipkart_df.shape

In [ ]:
df = pd.concat([amazon_df, flipkart_df], axis=0)   # `keys` argument를 활용하셔서 site를 구분하셔도 됩니다
df.shape

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['price'] = df['price'].astype(float)

df.set_index('date', inplace=True)
df = df.loc[:"2017-12-31"]   # 비록 DatetimeIndex이지만, 날짜를 문자열 string으로 표현하여 loc을 이용한 range indexing이 가능합니다.

df.rename(columns={'title_x':'name', 'title_y':'title'}, inplace=True)

df['price_grp'] = pd.cut(df['price'], [0, 5000, 15000, 200000], labels=["저가", "중가", "고가"])

In [ ]:
df.head()

## 전체제품 기간별 제품 평균가격

In [ ]:
# 아래에서 보이시는 것처럼 groupby()의 인자에는 반드시 'column명'이 아니어도 됩니다.
# 아래 예제처럼 df라는 object로부터 추출할 수 있는(여기서는 df.index에서 추출한) categorical 변수들을 사용해도 됩니다.
df.groupby([df.index.year, df.index.quarter]).agg({'price':'mean'})

In [ ]:
df.groupby([df.index.year, df.index.quarter]).agg({'price':'mean'}).plot(kind='bar')

In [ ]:
%matplotlib inline

In [ ]:
ax = df.resample("Q")['price'].mean().plot();

ax.set_title("기간별 제품 평균가격");
ax.set_xlabel("기 간");
ax.set_ylabel("가 격");

## 브랜드별 리뷰수

In [ ]:
df.groupby(['brand']).agg({'name':'count'})

In [ ]:
df.groupby(['brand']).agg({'name':'count'}).plot(kind='bar', figsize=(8,5));